<center><h1>Gunadi_Eben_Final_Project</h1></center>

Name: Eben Gunadi
<br>
Github Username: egunadi
<br>
USC ID: 3976793880

## 1. Text Classification

Import packages

In [7]:
import pandas as pd
from pathlib import Path
import glob
import re

### (a) Download the Reviews Data Set

In [3]:
# In this problem, we are trying to build a classifier to analyze the sentiment of reviews. You are provided with text data in two folders: one folder involves positive reviews, and one folder involves negative reviews.

positive_path = "../data/pos/"
negative_path = "../data/neg/"

positive_files = glob.glob(positive_path + "*.txt")
negative_files = glob.glob(negative_path + "*.txt")

In [40]:
file_id_pattern = re.compile("\d{3}")

positive_df = pd.DataFrame()

for file in positive_files:
    file_title = Path(file).stem
    file_id = int(file_id_pattern.search(file_title).group(0))

    with open(file, "r") as f:
        text = f.read()
        dict = {"id": file_id, "title": file_title, "text": text}

        df = pd.DataFrame(dict, index=[0])
        positive_df = pd.concat([positive_df, df], ignore_index=True)

In [41]:
negative_df = pd.DataFrame()

for file in negative_files:
    file_title = Path(file).stem
    file_id = int(file_id_pattern.search(file_title).group(0))

    with open(file, "r") as f:
        text = f.read()
        dict = {"id": file_id, "title": file_title, "text": text}

        df = pd.DataFrame(dict, index=[0])
        negative_df = pd.concat([negative_df, df], ignore_index=True)

In [42]:
positive_df = positive_df.set_index("id")
negative_df = negative_df.set_index("id")

In [46]:
print(positive_df.shape)
print(negative_df.shape)

(1000, 2)
(1000, 2)


In [47]:
positive_df.head()

,title,text
id,,
839,cv839_21467,assume nothing . \nthe phrase is perhaps one o...
34,cv034_29647,plot : derek zoolander is a male model . \nhe ...
908,cv908_16009,i actually am a fan of the original 1961 or so...
748,cv748_12786,a movie that's been as highly built up as the ...
253,cv253_10077,""" good will hunting "" is two movies in one : ..."


In [48]:
negative_df.head()

,title,text
id,,
676,cv676_22202,bad . bad . \nbad . \nthat one word seems to p...
839,cv839_22807,isn't it the ultimate sign of a movie's cinema...
155,cv155_7845,""" gordy "" is not a movie , it is a 90-minute-..."
465,cv465_23401,disconnect the phone line . \ndon't accept the...
398,cv398_17047,when robert forster found himself famous again...


### (b) Data Exploration and Pre-processing

#### (i) Binary Encoding

In [ ]:
# You can use binary encoding for the sentiments , i.e y = 1 for positive sentiments and y = −1 for negative sentiments.

#### (ii) Data Cleaning

In [ ]:
# The data are pretty clean. Remove the punctuation and numbers from the data.

#### (iii) Separate Test and Train Data

In [ ]:
# The name of each text file starts with cv number. Use text files 0-699 in each class for training and 700-999 for testing.

#### (iv) Unique Words

In [ ]:
# Count the number of unique words in the whole dataset (train + test) and print it out.

#### (v) Review Length

In [ ]:
# Calculate the average review length and the standard deviation of review lengths. Report the results.

#### (vi) Histogram

In [ ]:
# Plot the histogram of review lengths.

#### (vii) Tokenization

In [ ]:
# To represent each text (= data point), there are many ways. In NLP/Deep Learning terminology, this task is called tokenization. It is common to represent text using popularity/ rank of words in text. The most common word in the text will be represented as 1, the second most common word will be represented as 2, etc. Tokenize each text document using this method.

#### (viii) Review Length Threshold

In [ ]:
# Select a review length L that 70% of the reviews have a length below it. If you feel more adventurous, set the threshold to 90%.

#### (ix) Zero-pad Review Length

In [ ]:
# Truncate reviews longer than L words and zero-pad reviews shorter than L so that all texts (= data points) are of length L.

### (c) Word Embeddings

#### (i) Generate Word Embeddings

In [ ]:
# One can use tokenized text as inputs to a deep neural network. However, a recent breakthrough in NLP suggests that more sophisticated representations of text yield better results. These sophisticated representations are called word embeddings. “Word embedding is a term used for representation of words for text analysis, typically in the form of a real-valued vector that encodes the meaning of the word such that the words that are closer in the vector space are expected to be similar in meaning.” Most deep learning modules (including Keras) provide a convenient way to convert positive integer representations of words into a word embedding by an “Embedding layer.” The layer accepts arguments that define the mapping of words into embeddings, including the maximum number of expected words also called the vocabulary size (e.g. the largest integer value). The layer also allows you to specify the dimension for each word vector, called the “output dimension.” We would like to use a word embedding layer for this project. Assume that we are interested in the top 5,000 words. This means that in each integer sequence that represents each document, we set to zero those integers that represent words that are not among the top 5,000 words in the document.5 If you feel more adventurous, use all the words that appear in this corpus. Choose the length of the embedding vector for each word to be 32. Hence, each document is represented as a 32 × L matrix.


#### (ii) Flatten the Matrix

In [ ]:
# Flatten the matrix of each document to a vector.

### (d) Multi-Layer Perceptron

#### (i) Train a MLP

In [ ]:
# Train a MLP with three (dense) hidden layers each of which has 50 ReLUs and one output layer with a single sigmoid neuron. Use a dropout rate of 20% for the first layer and 50% for the other layers. Use ADAM optimizer and binary cross entropy loss (which is equivalent to having a softmax in the output). To avoid overfitting, just set the number of epochs as 2. Use a batch size of 10.

#### (ii) Train and Test Accuracies

In [ ]:
# Report the train and test accuracies of this model.

### (e) One-Dimensional Convolutional Neural Network

Although CNNs are mainly used for image data, they can also be applied to text data, as text also has adjacency information. Keras supports one-dimensional convolutions and pooling by the Conv1D and MaxPooling1D classes respectively.

#### (i) Insert a Conv1D Layer

In [ ]:
# After the embedding layer, insert a Conv1D layer. This convolutional layer has 32 feature maps , and each of the 32 kernels has size 3, i.e. reads embedded word representations 3 vector elements of the word embedding at a time. The convolutional layer is followed by a 1D max pooling layer with a length and stride of 2 that halves the size of the feature maps from the convolutional layer. The rest of the network is the same as the neural network above.

#### (ii) Train and Test Accuracies

In [ ]:
# Report the train and test accuracies of this model.

### (f) Long Short-Term Memory Recurrent Neural Network 

The structure of the LSTM we are going to use is shown in the following figure.

![](LSTM_structure.png)

#### (i) Train the Model

In [ ]:
# Each word is represented to LSTM as a vector of 32 elements and the LSTM is followed by a dense layer of 256 ReLUs. Use a dropout rate of 0.2 for both LSTM and the dense layer. Train the model using 10-50 epochs and batch size of 10.

#### (ii) Train and Test Accuracies

In [ ]:
# Report the train and test accuracies of this model.